In [1]:
import keras_snn
from jax import numpy as jnp

/home/cameron/keras-snn/keras_snn/__init__.py:7: UserWarning: Keras backend not set. Defaulting to JAX.
  warnings.warn("Keras backend not set. Defaulting to JAX.")


In [ ]:
jnp.array([1.0, 2.0, 3.0])

In [2]:
lif = keras_snn.layers.LIF()

In [3]:
lif(jnp.array([1.0, 2.0, 3.0]))

XlaRuntimeError: INTERNAL: XLA requires ptxas version 11.8 or higher